In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import pyshark
import os
from scipy.signal import savgol_filter
import math

<ipython-input-1-ed8a2480065c>:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [135]:
fname = 'los_1_iphone_tethering_in_lab'
df = pd.read_csv(fname + ".csv")
df.head(5)


# -------------

# ACCESS_POINT_MAC_ADDR = '00:1f:26:41:f0:00'
# ACCESS_POINT_MAC_ADDR = '3e:d1:9b:c7:a4:de'
# ACCESS_POINT_MAC_ADDR = 'e2:db:d1:d1:b0:4a'

unique_tx = df['tx_addr'].unique()
# print(unique_tx)
print("number of unique MAC addresses in packet capture: ", len(unique_tx))
print("MAC addresses in packet capture per value count:")
print(df.value_counts("tx_addr").to_string())

# -------------------

ACCESS_POINT_MAC_ADDR = 'e6:aa:ea:b7:4a:dd'

# --------------------

# filter by MAC address
pkts_from_ap_df = df[df['tx_addr'] == ACCESS_POINT_MAC_ADDR]
print(pkts_from_ap_df.shape)

# filtering: implement Savitzky-Golay method to preserve spikes and not excessively smooth vizualized patterns
# https://www.mathworks.com/help/curvefit/smoothing-data.html

data_rate_smooth = savgol_filter(pkts_from_ap_df['data_rate'], window_length=11, polyorder=3, mode="nearest")
signal_smooth = savgol_filter(pkts_from_ap_df['signal_dbm'], window_length=11, polyorder=3, mode="nearest")

pkts_from_ap_df.insert(3, "data_rate_smooth", data_rate_smooth, True)
pkts_from_ap_df.insert(7, "signal_smooth", signal_smooth, True)

pkts_from_ap_df.to_csv(fname + '_filtered_by_mac_addr.csv')


# -------------------

# df = file[["data_rate", "signal_dbm"]]
# df.plot(secondary_y = ['data_rate'])

df = pkts_from_ap_df

trace1 = go.Scatter(x=df['timestamp'],
                    y=df['data_rate'],
                    name='data_rate_plot',
                    mode='lines',
                    yaxis='y1')
trace2 = go.Scatter(x=df['timestamp'],
                    y=df['signal_dbm'],
                    name='signal_dbm_plot',
                    mode='lines',
                    yaxis='y1')
trace3 = go.Scatter(x=df['timestamp'],
                    y=df['data_rate_smooth'],
                    name='data_rate_smooth_plot',
                    mode='lines',
                    yaxis='y1')
trace4 = go.Scatter(x=df['timestamp'],
                    y=df['signal_smooth'],
                    name='signal_smooth_plot',
                    mode='lines',
                    yaxis='y1')
# trace3 = go.Bar(x=df['Timestamp'],
#                 y=df['ModelDiff'],
#                 name='Diff',
#                 yaxis='y2'
#                 )
data = [trace1, trace2, trace3, trace4]
layout = go.Layout(title='Plot of data rate and signal strength',
                   yaxis=dict(title='a'),
                   yaxis2=dict(title='b',
                               overlaying='y',
                               side='right'))
fig = go.Figure(data=data, layout=layout)
fig.write_html(fname + ".html")
fig.show()

number of unique MAC addresses in packet capture:  70
MAC addresses in packet capture per value count:
tx_addr
40:e3:d6:da:ba:23    752
40:e3:d6:da:ba:25    734
40:e3:d6:da:ba:20    717
40:e3:d6:da:ba:22    709
40:e3:d6:da:ba:21    702
e6:aa:ea:c9:ea:33    699
40:e3:d6:da:ba:24    656
40:e3:d6:da:ba:26    646
e6:aa:ea:b7:4a:dd    472
54:e4:3a:ec:fb:64    463
e6:aa:ea:c9:ea:1b    300
a6:e5:7c:28:b1:d3    159
b8:3a:5a:88:e6:e0     76
84:d4:7e:f2:42:26     61
84:d4:7e:f2:42:21     60
b8:3a:5a:88:e6:e1     49
84:d4:7e:f2:42:20     45
84:d4:7e:f2:42:22     44
84:d4:7e:f2:42:23     44
c0:c9:e3:4d:6e:bf     43
84:d4:7e:f2:42:24     42
84:d4:7e:f2:42:25     41
48:74:12:82:d8:d5     20
a4:e5:7c:28:b1:d3     18
22:a3:7f:87:e5:4e     11
78:8a:20:fa:27:f9      8
24:f6:77:10:02:c8      7
f0:51:ea:e5:56:1b      7
c4:23:60:66:07:8d      6
62:b0:c8:8e:5e:7b      5
84:d4:7e:d5:12:c0      4
ae:c4:df:e0:86:8d      3
9c:b6:d0:e8:0f:05      3
02:e7:af:de:b9:fc      3
de:0e:5d:2e:85:1e      2
f0:b6:1e:30:eb

In [125]:
# d_col = 'distance'
# f_col = 'frequency'
# fspl = ((4*math.pi*df[d_col]*df[f_col]) / c)**2